In [ ]:
library(Seurat)
library(ggplot2)

# Load data

In [ ]:
mouse <- readRDS("/projects/mludwig/DVC/output/Seurat_objs/mouse/mouse_Seurat_obj.rds")

# Subset neurons

In [ ]:
mouse.neurons <- subset(mouse, subset = SCT_snn_res.0.1 %in% c(0, 3, 4, 8, 9, 11))

# Run data normalization

In [ ]:
mouse.neurons <- SCTransform(mouse.neurons, verbose = F, method = "qpoisson")

# Run dimensionality reduction

In [ ]:
# PCA
mouse.neurons <- RunPCA(mouse.neurons, npcs = 100, verbose = F)

# Select number of PCs
ElbowPlot(mouse.neurons, ndims = 100)

# UMAP
dims <- 30
mouse.neurons <- RunUMAP(mouse.neurons, dims = 1:dims, n.neighbors = 50)

# Find clusters

In [ ]:
mouse.neurons <- FindNeighbors(mouse.neurons, dims = 1:dims, k.param = 50, verbose = F)
mouse.neurons <- FindClusters(mouse.neurons, resolution = 0.1, verbose = F)
mouse.neurons <- FindClusters(mouse.neurons, resolution = 1, verbose = F)

# Project labels

In [ ]:
# Neurons classified in Zhang et al.
zhang.neurons <- readRDS("/projects/mludwig/Ludwig-Diabetes-2021/data/Zhang/AP_neurons_Seurat_obj.rds")
zhang.neurons.no.NTS <- readRDS("/projects/mludwig/Ludwig-Diabetes-2021/data/Zhang/AP_neurons_no_NTS_Seurat_obj.rds")
zhang.neurons$origin <- "NTS"
zhang.neurons$origin[match(colnames(zhang.neurons.no.NTS), colnames(zhang.neurons))] <- zhang.neurons.no.NTS$seurat_clusters

zhang.neurons$cell.subtype <- ""
zhang.neurons$cell.subtype[which(zhang.neurons$SCT_snn_res.1.1 == 0)] <- "5_AP"
zhang.neurons$cell.subtype[which(zhang.neurons$SCT_snn_res.1.1 == 1)] <- "1_AP"
zhang.neurons$cell.subtype[which(zhang.neurons$SCT_snn_res.1.1 == 2)] <- "10_NTS"
zhang.neurons$cell.subtype[which(zhang.neurons$SCT_snn_res.1.1 == 3)] <- "6_AP"
zhang.neurons$cell.subtype[which(zhang.neurons$SCT_snn_res.1.1 == 4)] <- "2_AP"
zhang.neurons$cell.subtype[which(zhang.neurons$SCT_snn_res.1.1 == 5)] <- "8_NTS"
zhang.neurons$cell.subtype[which(zhang.neurons$SCT_snn_res.1.1 == 6)] <- "3_AP"
zhang.neurons$cell.subtype[which(zhang.neurons$SCT_snn_res.1.1 == 7)] <- "7_AP"
zhang.neurons$cell.subtype[which(zhang.neurons$SCT_snn_res.1.1 == 8)] <- "9_NTS"
zhang.neurons$cell.subtype[which(zhang.neurons$SCT_snn_res.1.1 == 9)] <- "4_AP"

# Neurons classified in Ludwig et al. 
ludwig.neurons <- readRDS("/projects/mludwig/DVC/output/Seurat_objs/AP_neurons_integrated_Seurat_obj_dims40.rds")

In [ ]:
# Project labels from Zhang et al. 
DefaultAssay(zhang.neurons) <- "SCT"
DefaultAssay(mouse.neurons) <- "SCT"

anchors <- FindTransferAnchors(reference = zhang.neurons, query = mouse.neurons, 
    dims = 1:dims, normalization.method = "SCT")
predictions <- TransferData(anchorset = anchors, refdata = zhang.neurons$cell.subtype, 
    dims = 1:dims)

mouse.neurons$zhang.predictions <- predictions$predicted.id
mouse.neurons$zhang.score <- predictions$prediction.score.max

In [ ]:
# Project label from Ludwig et al. 
DefaultAssay(ludwig.neurons) <- "integrated"
DefaultAssay(mouse.neurons) <- "SCT"

anchors <- FindTransferAnchors(reference = ludwig.neurons, query = mouse.neurons, 
    dims = 1:dims, normalization.method = "SCT")
predictions <- TransferData(anchorset = anchors, refdata = ludwig.neurons$cell.subtype2, 
    dims = 1:dims)

mouse.neurons$ludwig.predictions <- predictions$predicted.id
mouse.neurons$ludwig.score <- predictions$prediction.score.max

# Save

In [ ]:
mouse.neurons@meta.data <- mouse.neurons@meta.data[, c("nCount_RNA", "nFeature_RNA", 
                                                       "nCount_SCT", "nFeature_SCT",
                                                       "percent.mt", "pool", "hash.ID", 
                                                       "treatment", "run",
                                                       "SCT_snn_res.0.1", 
                                                       "zhang.predictions", "zhang.score",
                                                       "ludwig.predictions", "ludwig.score")]
saveRDS(mouse.neurons, 
        file = "/projects/mludwig/DVC/output/Seurat_objs/mouse/mouse_neurons_Seurat_obj.rds")